<center><img src="https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/Rainforest%20Connection%20Species%20Audio%20Detection/banner.png" width="1000"></center>
<br>
<center><h1>Rainforest - Audio classification Tensorflow starter</h1></center>
<br>

#### References:
- [Simple audio recognition: Recognizing keywords](https://www.tensorflow.org/tutorials/audio/simple_audio)
- [RFCX: train resnet50 with TPU](https://www.kaggle.com/yosshi999/rfcx-train-resnet50-with-tpu)
- [Getting Started: Rainforest EDA with TFRecords](https://www.kaggle.com/jessemostipak/getting-started-rainforest-eda-with-tfrecords)

### Workflow diagram
```
1. Load TFRecords
2. Decode audio waveform
3. Crop audio waveform
4. Convert the waveform into spectrogram
4. Resize spectrogram
5. Turn grayscale spectrogram into RGB image
6. Feed to the model
```
 
<center><img src="https://raw.githubusercontent.com/dimitreOliveira/MachineLearning/master/Kaggle/Rainforest%20Connection%20Species%20Audio%20Detection/Rainforest%20diagram.jpg" width="1000"></center>

## Dependencies

In [ ]:
!pip install --quiet efficientnet

In [ ]:
import os, re, warnings, random
import numpy as np
import pandas as pd
import seaborn as sns
from IPython import display as Idisplay
from matplotlib import pyplot as plt
from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
import tensorflow.keras.layers as L
import tensorflow.keras.backend as K
from tensorflow.keras import optimizers, losses, metrics, Model
from tensorflow.keras.callbacks import EarlyStopping
import efficientnet.tfkeras as efn
import tensorflow_addons as tfa

def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

seed = 0
seed_everything(seed)
warnings.filterwarnings('ignore')

### Hardware configuration

In [ ]:
# TPU or GPU detection
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print(f'Running on TPU {tpu.master()}')
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

AUTO = tf.data.experimental.AUTOTUNE
REPLICAS = strategy.num_replicas_in_sync
print(f'REPLICAS: {REPLICAS}')

# Model parameters

In [ ]:
BATCH_SIZE = 16 * REPLICAS
LEARNING_RATE = 1e-3 * REPLICAS
EPOCHS = 15
HEIGHT = 224
WIDTH = 512
CHANNELS = 3
N_CLASSES = 24
ES_PATIENCE = 3
TTA_STEPS = 6 # Do TTA if > 0 

# Load data

We have two different `.csv` files, `train_tp.csv` has the information of all true positive species labels and `train_fp.csv` has the information of all false positive species labels.

In [ ]:
def count_data_items(filenames):
    n = [int(re.compile(r'-([0-9]*)\.').search(filename).group(1)) for filename in filenames]
    return np.sum(n)


database_base_path = '/kaggle/input/rfcx-species-audio-detection/'
train_fp = pd.read_csv(f'{database_base_path}train_fp.csv')
train_tp = pd.read_csv(f'{database_base_path}train_tp.csv')

print(f'Train false positive samples: {len(train_fp)}')
display(train_fp.head())
print(f'Train true positive samples: {len(train_tp)}')
display(train_tp.head())

GCS_PATH = KaggleDatasets().get_gcs_path('rfcx-species-audio-detection')

TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/tfrecords/train/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/tfrecords/test/*.tfrec')

NUM_TRAINING_SAMPLES = count_data_items(TRAINING_FILENAMES)
NUM_TEST_SAMPLES = count_data_items(TEST_FILENAMES)

print(f'GCS: train samples: {NUM_TRAINING_SAMPLES}')
print(f'GCS: test samples: {NUM_TEST_SAMPLES}')

## Auxiliary functions

In [ ]:
# Datasets utility functions
def decode_audio(audio_binary):
    """
        Decode a 16-bit PCM WAV file to a float tensor.
    """
    audio, _ = tf.audio.decode_wav(audio_binary)
    return tf.squeeze(audio, axis=-1)
    return audio

def string_split_semicolon(column):
    split_labels_sc = tf.strings.split(column, sep=';')
    return split_labels_sc

def string_split_comma(column):
    split_labels_c = tf.strings.split(column, sep=',')
    return split_labels_c

def get_label_info(label_info):
    first_split = string_split_semicolon(label_info)
    remove_quotes = tf.strings.regex_replace(first_split, '"', "")
    label_info = string_split_comma(remove_quotes)
    return label_info

def get_spectrogram(waveform, padding=False, min_padding=48000):
    """
        Transforms a 'waveform' into a 'spectrogram', adding padding if needed.
    """
    waveform = tf.cast(waveform, tf.float32)
    if padding:
        # Padding for files with less than {min_padding} samples
        zero_padding = tf.zeros([min_padding] - tf.shape(waveform), dtype=tf.float32)
        # Concatenate audio with padding so that all audio clips will be of the same length
        waveform = tf.concat([waveform, zero_padding], 0)
    spectrogram = tf.signal.stft(waveform, frame_length=2048, frame_step=512, fft_length=2048)
    spectrogram = tf.abs(spectrogram)
    return spectrogram
    
def get_spectrogram_tf(example):
    """
        Transforms a 'waveform' tensor into a 'spectrogram'.
        Applied to a Tensorflow dataset.
    """
    audio = example['audio_wav']
    spectrogram = get_spectrogram(audio)
    spectrogram = tf.expand_dims(spectrogram, -1)
    example['audio_wav'] = spectrogram
    return example

def prepare_sample(example):
    """
        1. Resize samples to the expected size.
        2. Convert gray scales (1 channel) images to RGB (3 channels) format.
    """
    sample = example['audio_wav']
    sample = tf.image.resize(sample, [HEIGHT, WIDTH])
    sample = tf.image.grayscale_to_rgb(sample)
    example['audio_wav'] = sample
    return example

def crop_audio(audio, tmin, tmax, crop_size=10, sample_rate=48000, max_size=60):
    """
        Crops a 'waveform' file to have {crop_size} size given, {tmin}, {tmax}, {sample_rate} and {max_size}.
    """
    label_size = tmax - tmin
    
    if label_size >= crop_size: # No padding needed
        cut_min = tmin * sample_rate
        cut_max = (tmin + crop_size) * sample_rate
    else: # Needs padding
        if tmin <= (max_size - crop_size): # Pad at the end
            cut_min = tmin * sample_rate
            cut_max = (tmin + crop_size) * sample_rate
        else: # Pad at the beginning
            cut_min = (tmin - crop_size) * sample_rate
            cut_max = tmax * sample_rate
    
    # Casting tensors
    cut_min = tf.cast(cut_min, tf.int32)
    cut_max = tf.cast(cut_max, tf.int32)
    cut_size = tf.cast((crop_size*sample_rate), tf.int32)
    
    audio = audio[cut_min:cut_max] # croping the audio
    audio = audio[:cut_size] # making sure it has the max size
    
    audio = tf.reshape(audio, [cut_size]) # making sure it has the expected shape
    return audio

def random_crop_audio(audio, crop_size=10, sample_rate=48000, max_size=60):
    """
        Randomly crops a 'waveform' file to have {crop_size} size given, {sample_rate} and {max_size}.
    """
    start = tf.random.uniform([], minval=0, 
                              maxval=(max_size - crop_size), 
                              dtype=tf.int32)
    cut_min = start * sample_rate
    cut_max = (start + crop_size) * sample_rate
    
    # Casting tensors
    cut_min = tf.cast(cut_min, tf.int32)
    cut_max = tf.cast(cut_max, tf.int32)
    cut_size = tf.cast((crop_size*sample_rate), tf.int32)
    
    audio = audio[cut_min:cut_max] # croping the audio
    audio = audio[:cut_size] # making sure it has the max size
    
    audio = tf.reshape(audio, [cut_size]) # making sure it has the expected shape
    return audio

def read_tfrecord(example, labeled=True, inference=False):
    """
        1. Parse data based on the 'TFREC_FORMAT' map.
        2. Decode PCM WAV file.
        3. Break down the information from 'label_info' into other features.
        4. Crop the 'audio' waveform if needed.
        5. Returns the features as a dictionary.
    """
    TFREC_FORMAT = {
        'audio_wav': tf.io.FixedLenFeature([], tf.string), 
        'recording_id': tf.io.FixedLenFeature([], tf.string), 
        'label_info': tf.io.FixedLenFeature([], tf.string, default_value='-1,-1,0,0,0,0,1'), 
    }
        
    example = tf.io.parse_single_example(example, TFREC_FORMAT)
    audio = decode_audio(example['audio_wav'])
    
    # Break down 'label_info' into the data columns
    label_info = get_label_info(example['label_info'])
    species_id = tf.strings.to_number(tf.gather_nd(label_info, [0, 0]), tf.int32)
#     songtype_id = tf.strings.to_number(tf.gather_nd(label_info, [0, 1]), tf.int32)
    tmin = tf.strings.to_number(tf.gather_nd(label_info, [0, 2]))
#     fmin = tf.strings.to_number(tf.gather_nd(label_info, [0, 3]))
    tmax = tf.strings.to_number(tf.gather_nd(label_info, [0, 4]))
#     fmax = tf.strings.to_number(tf.gather_nd(label_info, [0, 5]))
    is_tp = tf.strings.to_number(tf.gather_nd(label_info, [0, 6]), tf.int32)

    if labeled:
        audio = crop_audio(audio, tmin, tmax)
    if inference:
        audio = random_crop_audio(audio)
        
    features = {'audio_wav': audio, 
                'recording_id': example['recording_id'], 
                'species_id': species_id, 
                'is_tp': is_tp
               }
    return features

def load_dataset(filenames, labeled=True, ordered=False, inference=False):
    """
        Load and parse the TFRecords.
    """
    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False
        dataset = tf.data.Dataset.list_files(filenames)
        dataset = dataset.interleave(tf.data.TFRecordDataset, num_parallel_calls=AUTO)
    else:
        dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO)
    dataset = dataset.with_options(ignore_order)    
    dataset = dataset.map(lambda x: read_tfrecord(x, labeled=labeled, inference=inference), num_parallel_calls=AUTO)
    return dataset

def conf_output(sample, labeled=True):
    """
        Configure the output of the dataset.
    """
    output = ({'input_audio': sample['audio_wav']}, sample['species_id'])
    return output

def get_dataset(filenames, labeled=True, ordered=False, repeated=False, inference=False):
    """
        1. Load TFRecord files, parse and generate features (waveform and meta-data).
        2. Filter the dataset to contain only true positive samples.
        3. Create 'spectrogram' from the 'waveform'.
        4. Prepare image for the model.
        5. Configure data to have the expected output format.
        6. Apply Tensorflow data functions to optimize training.
        
        Returns a Tensorflow dataset ready for training or inference.
    """
    dataset = load_dataset(filenames, labeled=labeled, inference=inference)
    
    if labeled:
        dataset = dataset.filter(_filtterTP)
    
    dataset = dataset.map(get_spectrogram_tf, num_parallel_calls=AUTO)
    dataset = dataset.map(prepare_sample, num_parallel_calls=AUTO)
    dataset = dataset.map(lambda x: conf_output(x, labeled=labeled), num_parallel_calls=AUTO)
    
    if not ordered:
        dataset = dataset.shuffle(256)
    if repeated:
        dataset = dataset.repeat()
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO)
    return dataset

def _filtterTP(x):
    return x['is_tp'] == 1

In [ ]:
# Visualization utility functions
def plot_spectrogram(spectrogram, ax):
    # Convert to frequencies to log scale and transpose so that the time is represented in the x-axis (columns).
    log_spec = np.log(spectrogram.T)
    height = log_spec.shape[0]
    X = np.arange(spectrogram.shape[0])
    Y = range(height)
    ax.pcolormesh(X, Y, log_spec)
    
def display_waveforms(ds, n_rows=3, n_cols=3, figsize=(20, 16)):
    n = n_rows*n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    for i, sample in enumerate(ds.take(n)):
        r = i // n_cols
        c = i % n_cols
        ax = axes[r][c]
        ax.plot(sample['audio_wav'].numpy())
        ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
        label = sample['species_id'].numpy()
        recording_id = sample['recording_id'].numpy().decode()
        ax.set_title(f'{recording_id} - {label}')
    plt.show()
    
def display_spectrograms(ds, n_rows=3, n_cols=3, figsize=(20, 16)):
    n = n_rows*n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    for i, sample in enumerate(ds.take(n)):
        r = i // n_cols
        c = i % n_cols
        ax = axes[r][c]
        plot_spectrogram(np.squeeze(sample['audio_wav'].numpy()), ax)
        label = sample['species_id'].numpy()
        recording_id = sample['recording_id'].numpy().decode()
        ax.set_title(f'{recording_id} - {label}')
    plt.show()
    
def inspect_preds(features, labels, preds, n_rows=3, n_cols=2, figsize=(20, 16)):
    n = n_rows*n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    for i, (feature, label, pred) in enumerate(zip(features, labels, preds)):
        r = i // n_cols
        c = i % n_cols
        ax = axes[r][c]
        plot_spectrogram(np.squeeze(feature), ax)
        if pred == label:
            color = 'black'
            title = f'{pred} [True]'
        else:
            color = 'red'
            title = f'{pred} [False, should be {label}]'
        ax.set_title(title, fontsize=14, color=color)
    plt.show()
        
def display_waveforms_audio_spectrogram(ds, n_samples=1, sample_rate=48000):
    for sample in ds.take(n_samples):
        waveform = sample['audio_wav']
        label = sample['species_id'].numpy()
        recording_id = sample['recording_id'].numpy().decode()
        spectrogram = get_spectrogram(waveform)

        print(f'Name: {recording_id}')
        print(f'Label: {label}')
        print(f'Waveform shape: {waveform.shape}')
        print(f'Spectrogram shape: {spectrogram.shape}')
        print(f'Audio playback')
        Idisplay.display(Idisplay.Audio(waveform, rate=sample_rate))
        
        fig, axes = plt.subplots(2, figsize=(12, 8))
        timescale = np.arange(waveform.shape[0])
        axes[0].plot(timescale, waveform.numpy())
        axes[0].set_title('Waveform')
        axes[0].set_xlim([0, waveform.shape[0]])
        plot_spectrogram(spectrogram.numpy(), axes[1])
        axes[1].set_title('Spectrogram')
        plt.show()
        
def inspect_preds(features, labels, preds, n_rows=3, n_cols=2, figsize=(20, 16)):
    n = n_rows*n_cols
    fig, axes = plt.subplots(n_rows, n_cols, figsize=figsize)
    for i, (feature, label, pred) in enumerate(zip(features, labels, preds)):
        r = i // n_cols
        c = i % n_cols
        ax = axes[r][c]
        
        feature = tf.image.rgb_to_grayscale(feature).numpy()
        plot_spectrogram(np.squeeze(feature), ax)
        if pred == label:
            color = 'black'
            title = f'{pred} [True]'
        else:
            color = 'red'
            title = f'{pred} [False, should be {label}]'
        ax.set_title(title, fontsize=14, color=color)
    plt.show()
    
# Model evaluation
def plot_metrics(history):
    fig, axes = plt.subplots(2, 1, sharex='col', figsize=(20, 8))
    axes = axes.flatten()
    
    axes[0].plot(history['loss'], label='Train loss')
    axes[0].plot(history['val_loss'], label='Validation loss')
    axes[0].legend(loc='best', fontsize=16)
    axes[0].set_title('Loss')
    axes[0].axvline(np.argmin(history['loss']), linestyle='dashed')
    axes[0].axvline(np.argmin(history['val_loss']), linestyle='dashed', color='orange')
    
    axes[1].plot(history['sparse_categorical_accuracy'], label='Train accuracy')
    axes[1].plot(history['val_sparse_categorical_accuracy'], label='Validation accuracy')
    axes[1].legend(loc='best', fontsize=16)
    axes[1].set_title('Accuracy')
    axes[1].axvline(np.argmax(history['sparse_categorical_accuracy']), linestyle='dashed')
    axes[1].axvline(np.argmax(history['val_sparse_categorical_accuracy']), linestyle='dashed', color='orange')

    plt.xlabel('Epochs', fontsize=16)
    sns.despine()
    plt.show()

# Reading audio files

The audio file will initially be read as a binary file, which you'll want to convert into a numerical tensor.

To load an audio file, you will use [tf.audio.decode_wav](https://www.tensorflow.org/api_docs/python/tf/audio/decode_wav), which returns the WAV-encoded audio as a Tensor and the sample rate.

A WAV file contains time series data with a set number of samples per second. Each sample represents the amplitude of the audio signal at that specific time. In a 16-bit system, the values range from -32768 to 32767. The sample rate for this dataset is 16kHz. Note that [tf.audio.decode_wav](https://www.tensorflow.org/api_docs/python/tf/audio/decode_wav) will normalize the values to the range [-1.0, 1.0].

#### Let's examine a few audio waveforms with their corresponding labels.

In [ ]:
train_waveform_ds = load_dataset(TRAINING_FILENAMES)

display_waveforms(train_waveform_ds)

# Spectrogram

You'll convert the waveform into a spectrogram, which shows frequency changes over time and can be represented as a 2D image. This can be done by applying the short-time Fourier transform (STFT) to convert the audio into the time-frequency domain.

A Fourier transform ([tf.signal.fft](https://www.tensorflow.org/api_docs/python/tf/signal/fft)) converts a signal to its component frequencies, but loses all time information. The STFT ([tf.signal.stft](https://www.tensorflow.org/api_docs/python/tf/signal/stft)) splits the signal into windows of time and runs a Fourier transform on each window, preserving some time information, and returning a 2D tensor that you can run standard convolutions on.

STFT produces an array of complex numbers representing magnitude and phase. However, you'll only need the magnitude for this tutorial, which can be derived by applying [tf.abs](https://www.tensorflow.org/api_docs/python/tf/math/abs) on the output of [tf.signal.stft](https://www.tensorflow.org/api_docs/python/tf/signal/stft).

Choose `frame_length` and `frame_step` parameters such that the generated spectrogram "image" is almost square. For more information on STFT parameters choice, you can refer to [this video](https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe) on audio signal processing.

You also want the waveforms to have the same length, so that when you convert it to a spectrogram image, the results will have similar dimensions. This can be done by simply zero padding the audio clips that are shorter than one second.

Next, you will explore the data. Compare the waveform, the spectrogram and the actual audio of a few examples from the dataset.

In [ ]:
display_waveforms_audio_spectrogram(train_waveform_ds, n_samples=3)

Examine the spectrogram "images" for different samples of the datasets.

In [ ]:
train_spectrogram_ds = train_waveform_ds.map(get_spectrogram_tf, num_parallel_calls=AUTO)

display_spectrograms(train_spectrogram_ds)

# Model

In [ ]:
def model_fn(input_shape, N_CLASSES):
    inputs = L.Input(shape=input_shape, name='input_audio')
    base_model = efn.EfficientNetB0(input_tensor=inputs, 
                                    include_top=False, 
                                    weights='noisy-student',
                                    drop_connect_rate=0.4)

    x = L.GlobalAveragePooling2D()(base_model.output)
    x = L.Dropout(.5)(x)
    output = L.Dense(N_CLASSES, activation='softmax', name='output')(x)
    
    model = Model(inputs=inputs, outputs=output)

    return model

# Training

In [ ]:
oof_pred = []; oof_labels = []
idxT, idxV = train_test_split(range(32), test_size=0.2, random_state=seed)

# Create train and validation sets
TRAIN_FILENAMES = tf.io.gfile.glob([GCS_PATH + '/tfrecords/train/%.2i*.tfrec' % x for x in idxT])
VALID_FILENAMES = tf.io.gfile.glob([GCS_PATH + '/tfrecords/train/%.2i*.tfrec' % x for x in idxV])
np.random.shuffle(TRAINING_FILENAMES)
ct_train = count_data_items(TRAIN_FILENAMES)
ct_valid = count_data_items(VALID_FILENAMES)
steps_per_epoch = 32 #(ct_train // BATCH_SIZE)

print(f'TRAIN: {idxT} | {ct_train} samples')
print(f'VALID: {idxV} | {ct_valid} samples')

## MODEL
with strategy.scope():
    model = model_fn((None, None, CHANNELS), N_CLASSES)

    model.compile(optimizer=tfa.optimizers.RectifiedAdam(lr=LEARNING_RATE, 
                                                         min_lr=1e-8, 
                                                         total_steps=int(steps_per_epoch*EPOCHS), 
                                                         warmup_proportion=0.3), 
                  loss=losses.SparseCategoricalCrossentropy(), 
                  metrics=[metrics.SparseCategoricalAccuracy()])

model_path = f'model.h5'
es = EarlyStopping(monitor='val_sparse_categorical_accuracy', mode='max', 
                   patience=ES_PATIENCE, restore_best_weights=True, verbose=1)

## TRAIN
history = model.fit(x=get_dataset(TRAIN_FILENAMES, labeled=True, ordered=False, repeated=True), 
                    validation_data=get_dataset(VALID_FILENAMES, labeled=True, ordered=True, repeated=False), 
                    steps_per_epoch=steps_per_epoch, 
                    epochs=EPOCHS, 
                    callbacks=[es], 
                    verbose=2).history

# Save last model weights
model.save_weights(model_path)

# OOF predictions
ds_valid = get_dataset(VALID_FILENAMES, labeled=True, ordered=True, repeated=False)
oof_labels.append([target.numpy() for x, target in iter(ds_valid.unbatch())])
x_oof = ds_valid.map(lambda x, target: x)
oof_pred.append(np.argmax(model.predict(x_oof), axis=-1))

## RESULTS
print(f"#### Accuracy = {np.max(history['val_sparse_categorical_accuracy']):.3f}")

## Model loss graph

In [ ]:
plot_metrics(history)

# Model evaluation

Now we can evaluate the performance of the model, first, we can evaluate the usual metrics like, `accuracy`, `precision`, `recall`, and `f1-score`, `scikit-learn` provides the perfect function for this `classification_report`.

## OOF metrics

In [ ]:
y_true = np.concatenate(oof_labels)
y_pred = np.concatenate(oof_pred)

print(classification_report(y_true, y_pred))

# Confusion matrix

Let's also take a look at the confusion matrix, this will give us an idea about what classes the model is mixing or having a hard time.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 12))
cfn_matrix = confusion_matrix(y_true, y_pred)
cfn_matrix = (cfn_matrix.T / cfn_matrix.sum(axis=1)).T
df_cm = pd.DataFrame(cfn_matrix)
ax = sns.heatmap(df_cm, cmap='Blues', annot=True, fmt='.2f', linewidths=.5)
plt.show()

# Visualize predictions

Finally, it is a good practice to always inspect some of the model's prediction by looking at the data, this can give an idea if the model is getting some predictions wrong because the data is really hard, or if it is because the model is actually bad.

In [ ]:
train_dataset = get_dataset(TRAINING_FILENAMES, ordered=True)

for features, label in train_dataset.take(1):
    preds = np.argmax(model.predict(features['input_audio']), axis=-1)[:6]
    labels = label.numpy()[:6]
    batch_features = features['input_audio'].numpy()[:6]
    
inspect_preds(batch_features, labels, preds)

# Test set predictions

In [ ]:
test_size = count_data_items(TEST_FILENAMES)
test_ds = get_dataset(TEST_FILENAMES, labeled=False, ordered=True, repeated=True, inference=True)
ct_steps = TTA_STEPS * ((test_size/BATCH_SIZE) + 1)
x_test = test_ds.map(lambda features, recording_id: features['input_audio'])
test_preds = model.predict(x_test, steps=ct_steps, verbose=1)[:(test_size * TTA_STEPS)]
test_preds = np.mean(test_preds.reshape(test_size, TTA_STEPS, N_CLASSES, order='F'), axis=1)

names_test_ds = load_dataset(TEST_FILENAMES, labeled=False, ordered=True)
names = [features['recording_id'].numpy().decode('utf-8') for features in iter(names_test_ds)]

In [ ]:
submission = pd.DataFrame({'recording_id': names})
for column in range(N_CLASSES):
    submission[f's{column}'] = test_preds[:, column]

submission.to_csv('submission.csv', index=False)
display(submission.head(10))

# Predicted classes distribution

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(18, 8))
ax = sns.countplot(y=test_preds.argmax(axis=-1), palette='viridis')
ax.tick_params(labelsize=16)

plt.show()